# Initial Data Filtering - Extract from .xlsx, filter, and write to .csv

There's a massive amount of data in this .xlsx file (over 500,000 rows). It contains all LCA requests from the year 2014. My first goal is to extract it from Excel format, drop rows and columns that I won't need, and write it to a csv for easier access.

I originally planned to use the past 3 years of data, but that was simply too much for my little 13' Mac to handle. :(

In [1]:
import pandas as pd

f = open("2014_LCA.xlsx", "r")

print "Reading"  # This takes a while...
raw_df = pd.read_excel(f)
print "Done Reading"
f.close()

Reading
Done Reading


Thanks to the associated .doc file, I can determine which columns aren't going to be valuable for the analysis I have planned:

"VISA_CLASS" - These are all H1B  
"PW_SOURCE_1" - Wage source info  
"OTHER_WAGE_SOURCE_1" - Wage source info  
"YR_SOURCE_PUB_1" - Wage source info  
"PW_UNIT_2"  
"PW_SOURCE_2"  
"OTHER_WAGE_SOURCE_2"  
"YR_SOURCE_PUB_2"  

In addition, let's check the number of null values in each column to see if there are any very sparse columns to drop.

In [2]:
print raw_df.isnull().sum()

LCA_CASE_NUMBER                        0
STATUS                                 0
LCA_CASE_SUBMIT                        0
DECISION_DATE                          0
VISA_CLASS                             0
LCA_CASE_EMPLOYMENT_START_DATE         4
LCA_CASE_EMPLOYMENT_END_DATE           5
LCA_CASE_EMPLOYER_NAME                28
LCA_CASE_EMPLOYER_ADDRESS             42
LCA_CASE_EMPLOYER_CITY                27
LCA_CASE_EMPLOYER_STATE               46
LCA_CASE_EMPLOYER_POSTAL_CODE         41
LCA_CASE_SOC_CODE                     49
LCA_CASE_SOC_NAME                   4271
LCA_CASE_JOB_TITLE                     7
LCA_CASE_WAGE_RATE_FROM               36
LCA_CASE_WAGE_RATE_TO             416572
LCA_CASE_WAGE_RATE_UNIT               36
FULL_TIME_POS                          5
TOTAL_WORKERS                          3
LCA_CASE_WORKLOC1_CITY                49
LCA_CASE_WORKLOC1_STATE               49
PW_1                                  84
PW_UNIT_1                             77
PW_SOURCE_1     

It looks like the second work place and WAGE_RATE_TO columns are too sparse to work with, so I'll drop those. The full list of columns to drop is as follows:

In [3]:
unwanted_columns = ["VISA_CLASS", "LCA_CASE_WAGE_RATE_TO", "PW_SOURCE_1", "OTHER_WAGE_SOURCE_1", "YR_SOURCE_PUB_1", 
                    "LCA_CASE_WORKLOC2_CITY", "LCA_CASE_WORKLOC2_CITY", "LCA_CASE_WORKLOC2_STATE", 
                    "PW_2", "PW_UNIT_2", "PW_SOURCE_2","OTHER_WAGE_SOURCE_2", "YR_SOURCE_PUB_2"]

raw_df = raw_df.drop(unwanted_columns, axis=1)

The LCA_CASE_SOC_NAME column is missing plenty of items, but is directly linked to the SOC_CODE column preceding it, so I can probably fill in most of the missing values by comparing rows with the same SOC_CODE. Let's get a list of the codes for these missing names, then print out the values associated with those codes. As long as there's a non-null value associated with a code, I can fill in the other rows it appears in.

In [4]:
soc_null_codes = raw_df.LCA_CASE_SOC_CODE[raw_df.LCA_CASE_SOC_NAME.isnull()].unique()

code_name_list = []

for code in soc_null_codes:
    names = raw_df.LCA_CASE_SOC_NAME[raw_df.LCA_CASE_SOC_CODE == code].unique()
    if len(names) > 1: # If there's something other than NaN here...
        code_name_list.append([code, names])

print code_name_list

[]


Well, so much for that. I'll use dropna() to remove rows with any null values. The biggest hit will come from dropping these rows with no SOC_NAME, but we have 500,000+ data points and this represents less than 10% of the total.

In [5]:
raw_df = raw_df.dropna()

Next, I need to clean up the EMPLOYER_NAME column. I later found that many rows were being mis-grouped due to this (ex. "GOOGLE, INC." and "GOOGLE  INC." would count as separate employers). I'll try to go about doing this by first grouping the rows by EMPLOYER_ADDRESS and EMPLOYER_POSTAL_CODE, then standardizing the names in all of those groups (the assumption being that there is only one company per street address per zip code).

However, I first need to clean up the POSTAL_CODE column. In most cases, the value is a standard 5 number ZIP code, but there are some full 9-digit ones that should be truncated.

In [6]:
raw_df["LCA_CASE_EMPLOYER_POSTAL_CODE"] = [str(t)[:5] for t in raw_df["LCA_CASE_EMPLOYER_POSTAL_CODE"]]

grouped_by_location = raw_df.groupby(["LCA_CASE_EMPLOYER_ADDRESS", "LCA_CASE_EMPLOYER_POSTAL_CODE"])

print grouped_by_location.size()

LCA_CASE_EMPLOYER_ADDRESS       LCA_CASE_EMPLOYER_POSTAL_CODE
#1 CHILDREN'S WAY               72202                             1
#1 HIGHLANDER WAY               33853                             1
#1 SMOKY VALLEY MINE ROAD       89045                             2
#16-529 WEST 48TH ST            10019                             1
#2 JOHN BREWERS BAY             00802                             3
#2 SAINT ANTHONY'S WAY          62002                             1
#202, 5609 COLLEGE AVE          94618                             2
#233, 75 BROADWAY               94111                            22
#3 NATURAL RESOURCES DRIVE      72215                             3
#3 RESEARCH PARK DRIVE          63304                             1
#48 PR STATE ROAD 165 KM 1.2    00968                             1
#5 STATEHOUSE PLAZA, SUITE 500  72201                             1
#50 COUNTY ROAD 13              87013                             1
#6 CITYPLACE DRIVE              63141                 

Well, Yale seems to have proven that the street addresses and the ZIP codes are just as error prone as the company names (you can find a zip code with a letter in it, if you look hard enough). I'll go for the low-hanging fruit and deformat the EMPLOYER_NAME column for now. This should fix most of the issues with name mis-matches, but the key thing going forward will be to use functions such as .contains() rather than exact matches to perform searches.

In [7]:
raw_df["LCA_CASE_EMPLOYER_NAME"] = raw_df["LCA_CASE_EMPLOYER_NAME"].str.replace(",", "")  # remove ","
raw_df["LCA_CASE_EMPLOYER_NAME"] = raw_df["LCA_CASE_EMPLOYER_NAME"].str.replace(".", "")  # remove "."
raw_df["LCA_CASE_EMPLOYER_NAME"] = raw_df["LCA_CASE_EMPLOYER_NAME"].str.replace(" - ", "-")  # remove whitespace
raw_df["LCA_CASE_EMPLOYER_NAME"] = raw_df["LCA_CASE_EMPLOYER_NAME"].str.replace("  ", " ")  # remove double spaces

Now, let's check the STATUS column. This is one of the most important columns, displaying the status of each LCA.

In [8]:
grouped_by_status = raw_df.groupby("STATUS")

print grouped_by_status.size()

STATUS
CERTIFIED              451389
CERTIFIED-WITHDRAWN     36008
DENIED                  11768
INVALIDATED                 1
REJECTED                    2
WITHDRAWN               15874
dtype: int64


I'm not sure whether INVALIDATED means WITHDRAWN or DENIED, so I'll drop that row (it's only 1 row). REJECTED should clearly be DENIED, however.

I originally wanted to drop all the WITHDRAWN rows because they don't provide information about whether or not an LCA was approved, but I ultimately decided to keep them to better understand which companies and industries are applying for LCAs in the first place.

In [9]:
raw_df = raw_df[raw_df.STATUS != "INVALIDATED"]
raw_df = raw_df.replace("REJECTED", "DENIED")

Finally, let's take a last look at all the numeric columns and see if there are any values which are clearly wrong.

In [10]:
print raw_df.describe()

       LCA_CASE_WAGE_RATE_FROM  TOTAL_WORKERS          PW_1  \
count             5.150410e+05  515041.000000  5.150410e+05   
mean              9.243991e+04       1.956689  6.977173e+04   
std               1.021826e+07       5.867164  1.542514e+06   
min               7.010000e+00       1.000000  0.000000e+00   
25%               6.000000e+04       1.000000  5.222900e+04   
50%               7.000000e+04       1.000000  6.333600e+04   
75%               9.000000e+04       1.000000  7.974720e+04   
max               7.278873e+09    2013.000000  8.201323e+08   

       LCA_CASE_NAICS_CODE  
count         5.150410e+05  
mean          4.725340e+05  
std           1.610983e+06  
min           1.100000e+01  
25%           5.182100e+05  
50%           5.415110e+05  
75%           5.415110e+05  
max           8.129908e+08  


Clearly, there are errors in the TOTAL_WORKERS and PW_1 columns. There is a row requesting 2013 total workers (a misplaced date, most likely) and a row claiming a wage of $0.00. I'll drop these and take another look.

In [11]:
raw_df = raw_df[raw_df.PW_1 != 0]
raw_df = raw_df[raw_df.TOTAL_WORKERS != 2013]

print raw_df.describe()

       LCA_CASE_WAGE_RATE_FROM  TOTAL_WORKERS          PW_1  \
count             5.150380e+05  515038.000000  5.150380e+05   
mean              9.244013e+04       1.952788  6.977204e+04   
std               1.021829e+07       5.154741  1.542519e+06   
min               7.010000e+00       1.000000  7.010000e+00   
25%               6.000000e+04       1.000000  5.222900e+04   
50%               7.000000e+04       1.000000  6.333600e+04   
75%               9.000000e+04       1.000000  7.974780e+04   
max               7.278873e+09     200.000000  8.201323e+08   

       LCA_CASE_NAICS_CODE  
count         5.150380e+05  
mean          4.725353e+05  
std           1.610988e+06  
min           1.100000e+01  
25%           5.182100e+05  
50%           5.415110e+05  
75%           5.415110e+05  
max           8.129908e+08  


The PW_1 (Prevailing Wage Rate) column looks fine now, but there's still a value of 200 in TOTAL_WORKERS. Since the 75% of that column is still 1, it's clearly an outlier - let's take a look...

In [12]:
print str(raw_df["LCA_CASE_EMPLOYER_NAME"][raw_df.TOTAL_WORKERS > 50].unique())

[u'ORACLE AMERICA INC' u'SYNTEL CONSULTING INC' u'SYNTEL INC'
 u'ITELLIGENCE INC' u'CFA INSTITUTE' u'SYNOPSYS INC'
 u'IGATE TECHNOLOGIES INC' u'AMAZON CORPORATE LLC'
 u'QUALCOMM TECHNOLOGIES INC' u'WIPRO LIMITED'
 u'TATA CONSULTANCY SERVICES LIMITED' u'APPLE INC' u'ACCENTURE LLP'
 u'AMIRIT TECHNOLOGIES INC' u'RVS LINE LLC' u'BILINGUALS INC'
 u'JUNIPER NETWORKS (US) INC' u'JUNIPER NETWORKS INC'
 u'PRICEWATERHOUSECOOPERS LLP' u'NIIT TECHNOLOGIES LIMITED'
 u'PRICEWATERHOUSECOOPERS ADVISORY LLC' u'DETROIT EDUCATION & RESEARCH'
 u'CAYLEY AEROSPACE INC' u'PROGRAMMER RESOURCES INTERNATIONAL INC'
 u'PRESENCE SAINT FRANCIS HOSPITAL' u'WILLIAM BEAUMONT HOSPITAL'
 u'PRESENCE SAINT JOSEPH HOSPITAL-CHICAGO'
 u'SPRINGLEAF GENERAL SERVICES CORPORATION']


It seems that there's actually several companies requesting more than 50 workers at a time. Let's look at Oracle.

In [13]:
oracle = raw_df["LCA_CASE_EMPLOYER_NAME"]=="ORACLE AMERICA INC"

print "Unique TOTAL_WORKERS values from Oracle:", raw_df["TOTAL_WORKERS"][oracle].unique()

print "Total applications from Oracle:", raw_df["TOTAL_WORKERS"][oracle].count()

print "Total applications from Oracle where TOTAL_WORKERS == 20:",\
                raw_df["TOTAL_WORKERS"][oracle][raw_df["TOTAL_WORKERS"]==20].count()
    
print "Total applications from Oracle where TOTAL_WORKERS == 100:",\
                raw_df["TOTAL_WORKERS"][oracle][raw_df["TOTAL_WORKERS"]==100].count()

Unique TOTAL_WORKERS values from Oracle: [   1.  100.   10.    4.    5.   20.]
Total applications from Oracle: 1073
Total applications from Oracle where TOTAL_WORKERS == 20: 25
Total applications from Oracle where TOTAL_WORKERS == 100: 1


So TOTAL_WORKERS isn't an accurate metric, which is good to know. Going forward, I won't use this column.

In [14]:
raw_df = raw_df.drop("TOTAL_WORKERS", axis=1)

At this point, I'm ready to write this dataframe to a csv. There are some non-ASCII characters here, so I'll need to set the encoding to utf-8.

In [15]:
raw_df.to_csv("Prepped_LCA.csv", encoding="utf-8")